<h1><center>
Toxic Comments Kaggle Competition Classifying
</center></h1>
<h2><center>
Modeling the Data
</center></h2>
<h3><center>
Marcel Colvin 912033961
</center></h3>

In [21]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import re
import glob
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("training_data.csv")
test = pd.read_csv("test_data.csv")

In [3]:
len(train)
len(test)

153164

# MLP Classifier with w2v algorithm
### This had 95.35% accuracy

In [4]:
train["avg_word"] = train["avg_word"].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [5]:
test["avg_word"] = test["avg_word"].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [6]:
train_matrix = np.zeros((len(train),300), dtype="float64")
test_matrix = np.zeros((len(test),300), dtype="float64")
for i in range(len(train)):
    train_matrix[i] = train.iloc[i]["avg_word"].reshape((1,300))
for i in range(len(test)):
        test_matrix[i] = test.iloc[i]["avg_word"].reshape((1,300))

In [7]:
y_train = [train["toxic"], train["severe_toxic"], train["obscene"], train["threat"], train["insult"], train["identity_hate"]]

In [8]:
x_train = train["avg_word"].values

In [9]:
model = []
for i in range(6):
    m = MLPClassifier(hidden_layer_sizes=(30,30,30), random_state=5)
    model.append(m)

In [10]:
for i in range(6):
    model[i].partial_fit(train_matrix, y_train[i], [0,1])

In [11]:
result = test[['id']].copy() 

In [12]:
result['toxic'] = model[0].predict_proba(test_matrix)[:,1]
result['severe_toxic'] = model[1].predict_proba(test_matrix)[:,1]
result['obscene'] = model[2].predict_proba(test_matrix)[:,1]
result['threat'] = model[3].predict_proba(test_matrix)[:,1]
result['insult'] = model[4].predict_proba(test_matrix)[:,1]
result['identity_hate'] = model[5].predict_proba(test_matrix)[:,1]

In [13]:
result.to_csv("submission.csv", index = False)

## 95.09%

In [13]:
model = []
for i in range(6):
    m = LogisticRegression(C=10.0)
    model.append(m)

In [14]:
for i in range(6):
    model[i].fit(train_matrix, y_train[i])

In [15]:
result = test[['id']].copy() 

In [16]:
result['toxic'] = model[0].predict_proba(test_matrix)[:,1]
result['severe_toxic'] = model[1].predict_proba(test_matrix)[:,1]
result['obscene'] = model[2].predict_proba(test_matrix)[:,1]
result['threat'] = model[3].predict_proba(test_matrix)[:,1]
result['insult'] = model[4].predict_proba(test_matrix)[:,1]
result['identity_hate'] = model[5].predict_proba(test_matrix)[:,1]

In [17]:
result.to_csv("submission.csv", index = False)

# TFIDF Logistitc Regression, "comment_text" uncleaned
### This had 93.94% accuracy

In [18]:
tfidfvect = TfidfVectorizer(max_features=5000,stop_words='english')

In [19]:
X = tfidfvect.fit_transform(train["comment_text"])

In [17]:
testX = tfidfvect.transform(test["comment_text"])

In [16]:
y_train = [train["toxic"], train["severe_toxic"], train["obscene"], train["threat"], train["insult"], train["identity_hate"]]

In [18]:
model = []
for i in range(6):
    m = LogisticRegression(C=10.0)
    model.append(m)

In [19]:
for i in range(6):
    model[i].fit(X,y_train[i])

In [20]:
result = test[['id']].copy() 

In [21]:
result['toxic'] = model[0].predict_proba(testX)[:,1]
result['severe_toxic'] = model[1].predict_proba(testX)[:,1]
result['obscene'] = model[2].predict_proba(testX)[:,1]
result['threat'] = model[3].predict_proba(testX)[:,1]
result['insult'] = model[4].predict_proba(testX)[:,1]
result['identity_hate'] = model[5].predict_proba(testX)[:,1]

In [22]:
result.to_csv("submission.csv", index = False)

# Cleaning "comment_text" to run TFIDF Logistic Regression Again
### This had 96.30% accuracy

In [23]:
def text_cleaner(word):
    word = word.lower()
    word = re.sub(r"\'s", ' ', word)
    word = re.sub("<[^>]*>", " ", word)
    word = re.sub('\W', ' ', word)
    word = re.sub('\s+', ' ', word)
    word = word.strip(' ')
    return word

In [24]:
train["comment_text"] = train['comment_text'].map(lambda word : text_cleaner(word))
test["comment_text"] = test['comment_text'].map(lambda word : text_cleaner(word))

In [25]:
tfidfvect = TfidfVectorizer(max_features=5000,stop_words='english')

In [26]:
X = tfidfvect.fit_transform(train["comment_text"])

In [27]:
y_train = [train["toxic"], train["severe_toxic"], train["obscene"], train["threat"], train["insult"], train["identity_hate"]]

In [28]:
testX = tfidfvect.transform(test["comment_text"])

In [29]:
model = []
for i in range(6):
    m = LogisticRegression(C=10.0)
    model.append(m)

In [55]:
for i in range(6):
    model[i].fit(X,y_train[i])

TypeError: fit() takes 3 positional arguments but 4 were given

In [31]:
result = test[['id']].copy() 

In [32]:
result['toxic'] = model[0].predict_proba(testX)[:,1]
result['severe_toxic'] = model[1].predict_proba(testX)[:,1]
result['obscene'] = model[2].predict_proba(testX)[:,1]
result['threat'] = model[3].predict_proba(testX)[:,1]
result['insult'] = model[4].predict_proba(testX)[:,1]
result['identity_hate'] = model[5].predict_proba(testX)[:,1]

In [33]:
result.to_csv("submission.csv", index = False)

# MLPClassifier Quick
### 95.21% accuracy

In [53]:
model = []
for i in range(6):
    m = MLPClassifier(hidden_layer_sizes=(30,30,30), random_state=5)
    model.append(m)

In [56]:
for i in range(6):
    model[i].partial_fit(X,y_train[i], [0,1])

In [57]:
result = test[['id']].copy() 

In [58]:
result['toxic'] = model[0].predict_proba(testX)[:,1]
result['severe_toxic'] = model[1].predict_proba(testX)[:,1]
result['obscene'] = model[2].predict_proba(testX)[:,1]
result['threat'] = model[3].predict_proba(testX)[:,1]
result['insult'] = model[4].predict_proba(testX)[:,1]
result['identity_hate'] = model[5].predict_proba(testX)[:,1]

In [59]:
result.to_csv("submission.csv", index = False)

# HHH

In [34]:
path = '/home/marcel/Github/Toxic_comments/nyt-comments/comments/'
nyt_comments = pd.DataFrame()
for file_name in glob.glob(path+'*.csv'):
    nyt_comments = nyt_comments.append(pd.read_csv(file_name)[['commentBody', 'commentID', "reportAbuseFlag", 'userLocation', 'userID']])

/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsMarch2018.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsApril2018.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsMarch2017.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsApril2017.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsJan2018.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsMay2017.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsJan2017.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsFeb2018.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsFeb2017.csv


In [35]:
nyt_comments["commentBody"] = nyt_comments['commentBody'].map(lambda word : text_cleaner(word))

In [36]:
nyt_comments = nyt_comments.reset_index(drop = True)

In [37]:
test_nyt = tfidfvect.transform(nyt_comments["commentBody"])

In [41]:
result = nyt_comments[["commentID"]].copy()

In [42]:
result['toxic'] = model[0].predict_proba(test_nyt)[:,1]
result['severe_toxic'] = model[1].predict_proba(test_nyt)[:,1]
result['obscene'] = model[2].predict_proba(test_nyt)[:,1]
result['threat'] = model[3].predict_proba(test_nyt)[:,1]
result['insult'] = model[4].predict_proba(test_nyt)[:,1]
result['identity_hate'] = model[5].predict_proba(test_nyt)[:,1]

In [43]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [44]:
toxic_nyt_coms = nyt_comments.loc[np.where(result['toxic'] > .8)]["commentBody"].reset_index(drop = True)

In [45]:
x = [None]*len(toxic_nyt_coms)
for i in range(len(toxic_nyt_coms)):
     x[i] = sid.polarity_scores(toxic_nyt_coms[i])

In [46]:
toxic_comments = pd.DataFrame(toxic_nyt_coms)

In [47]:
toxic_comments["sent_anal"] = x

In [48]:
toxic_comments['neg'] = [d['neg'] for d in toxic_comments["sent_anal"]]

In [49]:
toxic_comments['neu'] = [d['neu'] for d in toxic_comments["sent_anal"]]

In [50]:
toxic_comments['pos'] = [d['pos'] for d in toxic_comments["sent_anal"]]

In [51]:
toxic_comments['compound'] = [d['compound'] for d in toxic_comments["sent_anal"]]

In [60]:
np.mean(toxic_comments["neg"])

0.24774395560840243

In [61]:
np.mean(toxic_comments["pos"])

0.08740048694864437

In [62]:
np.mean(toxic_comments["neu"])

0.6648578789423044

In [63]:
np.mean(toxic_comments["compound"])

-0.5204957533548517

In [66]:
x = [None]*len(nyt_comments)
for i in range(len(nyt_comments)):
     x[i] = sid.polarity_scores(nyt_comments.iloc[i]["commentBody"])

In [67]:
#nyt_comments["sent_anal"] = x
nyt_comments['neg'] = [d['neg'] for d in x]
nyt_comments['neu'] = [d['neu'] for d in x]
nyt_comments['pos'] = [d['pos'] for d in x]
nyt_comments['compound'] = [d['compound'] for d in x]

In [68]:
np.mean(nyt_comments["neg"])

0.09928626093795935

In [69]:
np.mean(nyt_comments["pos"])

0.11829671920689967

In [70]:
np.mean(nyt_comments["neu"])

0.782388060085638

In [71]:
np.mean(nyt_comments["compound"])

0.06153051704583903

In [72]:
nyt_comments.to_csv("nyt_commentsdf.csv", index = False)